## Instalar librerias

In [ ]:
!pip install -q ultralytics opencv-python shapely tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 62.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.1 MB/s eta 0:00:00


In [ ]:
!unzip -q MBD_sample.zip -d /content/MBD_sample

## Conversión a formato YOLOv8-Seg

In [ ]:
import cv2, glob, shutil, os, numpy as np, yaml, random
from shapely.geometry import Polygon
from pathlib import Path
from tqdm import tqdm

DATA_ROOT   = Path('/content/MBD_sample/MBD_sample')         # ajusta si tu ruta es otra
PNG_ROOT    = DATA_ROOT / 'png'
YOLO_ROOT   = DATA_ROOT / 'yolo'           # nueva estructura images/ + labels/
SPLITS      = ['train', 'val', 'test']

for split in SPLITS:
    (YOLO_ROOT / f'images/{split}').mkdir(parents=True, exist_ok=True)
    (YOLO_ROOT / f'labels/{split}').mkdir(parents=True, exist_ok=True)

def mask_to_polys(mask):
    """Devuelve lista de polígonos normalizados en formato [[x1,y1,x2,y2,...], ...]"""
    h, w = mask.shape
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    polys = []
    for cnt in contours:
        if cv2.contourArea(cnt) < 50:          # filtrar ruido
            continue
        cnt = cnt.squeeze()
        if cnt.ndim != 2 or len(cnt) < 3:
            continue
        # simplificar contorno para reducir nº vértices (tolerancia 1 px)
        cnt = cv2.approxPolyDP(cnt, 1, True).squeeze()
        norm = cnt.astype(float)
        norm[:, 0] /= w
        norm[:, 1] /= h
        polys.append(norm.flatten().tolist())
    return polys

for split in SPLITS:
    img_dir  = PNG_ROOT / split
    msk_dir  = PNG_ROOT / f'{split}_labels'
    for img_path in tqdm(sorted(glob.glob(str(img_dir / '*.png'))), desc=f'Converting {split}'):
        fname   = Path(img_path).stem
        dst_img = YOLO_ROOT / f'images/{split}/{fname}.jpg'  # YOLO acepta jpg o png
        dst_lbl = YOLO_ROOT / f'labels/{split}/{fname}.txt'

        # copiar imagen (o convertir a jpg para ahorrar espacio)
        img = cv2.imread(img_path)
        cv2.imwrite(str(dst_img), img)

        # procesar máscara
        mask = cv2.imread(str(msk_dir / f'{fname}.png'), cv2.IMREAD_GRAYSCALE)
        polys = mask_to_polys(mask)
        if not polys:                     # si imagen no contiene edificios
            continue
        with open(dst_lbl, 'w') as f:
            for poly in polys:
                f.write('0 ' + ' '.join(f'{p:.6f}' for p in poly) + '\n')

Converting test: 100%|██████████| 10/10 [00:01<00:00,  5.27it/s]


## YAML de configuración para Ultralytics

In [ ]:
yaml_path = YOLO_ROOT / 'mass_buildings.yaml'
yaml.dump({
    'path' : str(YOLO_ROOT),
    'train': 'images/train',
    'val'  : 'images/val',
    'test' : 'images/test',
    'nc'   : 1,
    'names': ['building']
}, open(yaml_path, 'w'))
print('Dataset YAML:\n', open(yaml_path).read())

Dataset YAML:
 names:
- building
nc: 1
path: /content/MBD_sample/MBD_sample/yolo
test: images/test
train: images/train
val: images/val



## Visualización de ejemplos etiquetados

In [ ]:
import matplotlib.pyplot as plt

def show_random(n=3, split='train'):
    imgs = sorted(glob.glob(str(YOLO_ROOT / f'images/{split}/*.jpg')))
    for path in random.sample(imgs, n):
        img = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)
        h, w = img.shape[:2]

        # leer polígonos y dibujar
        txt = Path(path.replace('images', 'labels').replace('.jpg', '.txt'))
        if txt.exists():
            with open(txt) as f:
                for line in f:
                    pts = np.array(list(map(float, line.strip().split()[1:])),
                                   dtype=np.float32).reshape(-1, 2)
                    pts[:, 0] *= w
                    pts[:, 1] *= h
                    cv2.polylines(img, [pts.astype(int)], isClosed=True,
                                  color=(255, 0, 0), thickness=1)
        plt.figure(figsize=(8,8)); plt.imshow(img); plt.axis('off'); plt.title(Path(path).stem)

show_random(n=1)


Output hidden; open in https://colab.research.google.com to view.

##  Inferencia BASELINE (sin ajuste) y métrica mAP-mask

In [ ]:
from ultralytics import YOLO
model_base = YOLO('yolov8n-seg.pt')

In [ ]:
metrics = model_base.val(
    data="/content/MBD_sample/MBD_sample/yolo/mass_buildings.yaml",        # ruta al YAML
    split="val",                         # o 'test'
    device='cpu'                             # GPU; usa 'cpu' si no tienes CUDA
)

Ultralytics 8.3.154 🚀 Python-3.11.13 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)


100%|██████████| 755k/755k [00:00<00:00, 13.7MB/s]

val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 724.1±248.3 MB/s, size: 1452.0 KB)



val: Scanning /content/MBD_sample/MBD_sample/yolo/labels/val... 4 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4/4 [00:00<00:00, 28.16it/s]

val: New cache created: /content/MBD_sample/MBD_sample/yolo/labels/val.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95):   0%|          | 0/1 [00:00<?, ?it/s]

WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:11<00:00, 11.42s/it]


                   all          4       4265     0.0621    0.00211     0.0313    0.00731     0.0621    0.00211     0.0312    0.00693
                person          4       4265     0.0621    0.00211     0.0313    0.00731     0.0621    0.00211     0.0312    0.00693
Speed: 7.7ms preprocess, 911.0ms inference, 0.0ms loss, 93.8ms postprocess per image
Results saved to runs/segment/val


In [ ]:
print("mAP-mask (IoU 0.5-0.95):", metrics.seg.map)     # mAP@[.5:.95]
print("mAP-mask50:",            metrics.seg.map50)    # mAP@0.5
print("mAP-mask75:",            metrics.seg.map75)    # mAP@0.75

mAP-mask (IoU 0.5-0.95): 0.006929924494628939
mAP-mask50: 0.031173724471988993
mAP-mask75: 0.0


## Fine-Tuning

In [ ]:
!yolo task=segment mode=train model=yolov8n-seg.pt data={yaml_path} epochs=10 imgsz=640 batch=8 name=MBD-buildings-FT

Ultralytics 8.3.154 🚀 Python-3.11.13 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/MBD_sample/MBD_sample/yolo/mass_buildings.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=MBD-buildings-FT3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=

In [ ]:
ft_model = YOLO('/content/runs/segment/MBD-buildings-FT3/weights/best.pt')

In [ ]:
metrics_ft = ft_model.val(
    data="/content/MBD_sample/MBD_sample/yolo/mass_buildings.yaml",        # ruta a tu YAML ⚠️ ajusta el path
    split="val",                         # o 'test' si prefieres
    device='cpu'                             # GPU; usa 'cpu' si no tienes CUDA
)

Ultralytics 8.3.154 🚀 Python-3.11.13 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
YOLOv8n-seg summary (fused): 85 layers, 3,258,259 parameters, 0 gradients, 12.0 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1799.8±272.8 MB/s, size: 1452.0 KB)


val: Scanning /content/MBD_sample/MBD_sample/yolo/labels/val.cache... 4 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4/4 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95):   0%|          | 0/1 [00:00<?, ?it/s]

WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:08<00:00,  8.65s/it]


                   all          4       4265     0.0383     0.0108     0.0242    0.00894      0.055     0.0155      0.028    0.00879
Speed: 2.2ms preprocess, 339.8ms inference, 0.0ms loss, 194.9ms postprocess per image
Results saved to runs/segment/val3


In [ ]:
print("mAP-mask (IoU 0.5-0.95):", metrics_ft.seg.map)     # mAP@[.5:.95]
print("mAP-mask50:",            metrics_ft.seg.map50)    # mAP@0.5
print("mAP-mask75:",            metrics_ft.seg.map75)    # mAP@0.75

mAP-mask (IoU 0.5-0.95): 0.008792758033167683
mAP-mask50: 0.02795395737546391
mAP-mask75: 0.005856519054340156
